In [1]:
from mlrl.runners.dqn_runner import DQNRun
from mlrl.experiments.procgen_dqn import make_procgen, create_rainbow_agent

pygame 2.1.0 (SDL 2.0.16, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
config = {
    'procgen_env_name': 'coinrun',
    'frame_stack': 4,
    'grayscale': False,
    'agent': 'rainbow',
    'learning_rate': 2.5e-4,
}

In [3]:
env_name = config.get('procgen_env_name', 'coinrun')
env = make_procgen(env_name, config)
# q_net, agent = create_rainbow_agent(env, config)

In [4]:
import tensorflow as tf
from tf_agents.utils import common
from tf_agents.drivers import py_driver
from tf_agents.agents.dqn.dqn_agent import DdqnAgent
from tf_agents.networks.q_network import QNetwork
from tf_agents.specs import tensor_spec
from tf_agents.networks.categorical_q_network import CategoricalQNetwork
from tf_agents.agents import CategoricalDqnAgent
from tf_agents.environments.py_environment import PyEnvironment

In [5]:
# categorical_q_net = CategoricalQNetwork(
#     tensor_spec.from_spec(env.observation_spec()),
#     tensor_spec.from_spec(env.action_spec()),
#     conv_layer_params=[(64, 8, 4), (64, 4, 2), (64, 3, 2)],
#     fc_layer_params=[512]
# )

In [6]:
# class RainbowQNet(tf.keras.Sequential):

#     def __init__(self, observation_shape, n_actions, name='RainbowQNet'):
#         super(RainbowQNet, self).__init__(name=name)

#         self.observation_shape = observation_shape
#         self.n_actions = n_actions

#         self.network = CategoricalQNetwork(
#             tensor_spec.TensorSpec(shape=observation_shape, dtype=tf.float64, name='observation'),
#             tensor_spec.BoundedTensorSpec(shape=(), dtype=tf.int64, minimum=0, maximum=n_actions - 1, name='action'),
#             conv_layer_params=[(64, 8, 4), (64, 4, 2), (64, 3, 2)],
#             fc_layer_params=[512]
#         )

#         self.add(self.network)

#         self(tf.keras.Input(shape=(1,) + tuple(observation_shape), dtype=tf.float64))
    
#     def get_config(self):
#         return {
#             'observation_shape': self.observation_shape,
#             'n_actions': self.n_actions
#         }
    
#     @classmethod
#     def from_config(cls, config):
#         return cls(**config)
    
#     def create_agent(self, env, config):
#         return CategoricalDqnAgent(
#             env.time_step_spec(),
#             env.action_spec(),
#             q_network=self,
#             optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
#             td_errors_loss_fn=tf.math.squared_difference,
#             gamma=0.99,
#             train_step_counter=tf.Variable(0)
#         )

In [7]:
from mlrl.networks.cnn_q_nets import RainbowQNet

In [8]:
observation_shape = env.observation_spec().shape
n_actions = env.action_spec().num_values
learning_rate = config.get('learning_rate', 2.5e-4)

rainbow_q_net = RainbowQNet(observation_shape, n_actions, learning_rate)

In [10]:
from mlrl.procgen import REWARD_BOUNDS as PROCGEN_REWARD_BOUNDS

if env_name not in PROCGEN_REWARD_BOUNDS:
    raise ValueError(f'Unknown reward bounds for procgen env: {env_name}')
r_min, r_max = PROCGEN_REWARD_BOUNDS[env_name]

config['r_min'] = r_min
config['r_max'] = r_max

rainbow_agent = rainbow_q_net.create_agent(env, config)

In [11]:
from mlrl.experiments.procgen_dqn import create_video_renderer

In [13]:
render_video = create_video_renderer(env_name, config)

In [17]:
from tf_agents.policies.py_tf_eager_policy import PyTFEagerPolicy
render_video(PyTFEagerPolicy(rainbow_agent.collect_policy, use_tf_function=True, batch_time_steps=False), 'video.mp4')

'video.mp4'

In [113]:
rainbow_q_net(env.current_time_step().observation);

In [166]:
rainbow_q_net.save('model.h5')

In [167]:
rainbow_reloaded = tf.keras.models.load_model('model.h5', custom_objects={'RainbowQNet': RainbowQNet})

In [172]:
rainbow_reloaded(env.current_time_step().observation)[0].shape

TensorShape([64, 15, 51])